In [1]:
import os
import numpy as np
import networkx as nx
from deap import base, creator, tools, algorithms

In [3]:
# Параметры
num_graphs = 1000
num_nodes = 30
pop_size = 100  # Размер популяции
num_generations = 50  # Количество поколений

In [5]:
# Директория для хранения файлов
script_dir = os.getcwd()
directory = os.path.join(script_dir, 'data')
os.makedirs(directory, exist_ok=True)
# Инициализация массива весовых матриц и "хороших" путей
weights_matrices = np.zeros((num_graphs, num_nodes, num_nodes))
good_paths = np.zeros((num_graphs, num_nodes), dtype=int)

In [6]:
# Создание функции для вычисления стоимости пути
def evaluate_path(path, graph):
    weight = sum(graph[path[i]][path[i+1]]['weight'] for i in range(len(path) - 1))
    weight += graph[path[-1]][path[0]]['weight']  # Замкнуть цикл
    return weight

In [7]:
# Генетический алгоритм для поиска пути
def find_good_path(graph):
    # Настройка параметров GA
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    toolbox = base.Toolbox()
    toolbox.register("indices", np.random.permutation, num_nodes)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("evaluate", evaluate_path, graph=graph)
    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    # Создание начальной популяции
    population = toolbox.population(n=pop_size)
    
    # Запуск алгоритма
    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations, verbose=False)
    
    # Находим лучший путь
    best_individual = tools.selBest(population, k=1)[0]
    return best_individual

TODO:сделать так что бы весь датасет был json файлом 

In [ ]:
# Генерация графов и их весовых матриц
for i in range(num_graphs):
    G = nx.complete_graph(num_nodes)
    
    for (u, v) in G.edges():
        weight = np.random.uniform(1, 10)
        G[u][v]['weight'] = weight
    
    weights_matrix = nx.to_numpy_array(G, weight='weight')
    weights_matrices[i] = weights_matrix
    
    # Нахождение "хорошего" пути с использованием генетического алгоритма
    best_path = find_good_path(G)
    good_paths[i] = best_path

In [9]:
# Проверка существования файла
if os.path.exists(os.path.join(directory,"good_paths.npy")):
    with open(os.path.join(directory,"good_paths.npy"), 'rb') as file:
        good_paths=np.load(file)
        type(good_paths)
else:
    print("Файл не найден.")

In [12]:
# Количество перестановок для каждого пути
n=len(good_paths[0])
num_permutations = n
num_nodes = good_paths.shape[1]  # Количество вершин (длина каждого пути)
noisy_paths = []

In [13]:
# Генерация зашумленных версий каждого пути
for path in good_paths:
    for i in range(1, num_permutations + 1):
        # Вычисляем количество перемешанных элементов
        num_swaps = (i * num_nodes) // num_permutations
        
        # Создаем зашумленный путь
        noisy_path = path.copy()
        swap_indices = np.random.choice(num_nodes, num_swaps, replace=False)
        
        
        # Выполняем перестановки
        for j in range(0, num_swaps - 1, 2):
            noisy_path[swap_indices[j]], noisy_path[swap_indices[j+1]] = noisy_path[swap_indices[j+1]], noisy_path[swap_indices[j]]
        
        # Добавляем зашумленный путь в список
        noisy_paths.append(noisy_path)

In [ ]:
# Преобразование списка в numpy массив
noisy_paths = np.array(noisy_paths)
np.save(os.path.join(directory,"noisy_paths.npy"), noisy_paths)